In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
import re
from random import randint
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pickle
import requests
import math
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import cfscrape
from lxml import etree

# Initiate webdriver

In [125]:
browser = webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver')

### Getting products links

In [3]:
products_collection = ['https://www.obi.de/gartenbau/terrassendielen/c/1464',
                       'https://www.obi.de/laminat-parkett-vinylboeden/laminat/c/514',
                       'https://www.obi.de/laminat-parkett-vinylboeden/musterboeden/c/2193',
                       'https://www.obi.de/laminat-parkett-vinylboeden/korkboeden/c/1174',
                       'https://www.obi.de/bauen/bauholz/c/169',
                       'https://www.obi.de/gartenbau/holzfliesen/c/1463',
                       'https://www.obi.de/moebelbau/arbeitsplatten/c/916',
                       'https://www.obi.de/moebelbau/regalboeden-moebelbauplatten/c/917']

In [6]:
header = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9',
          'Accept-Encoding': 'gzip, deflate, sdch',
          'Accept-Language' : 'nl-NL,nl;q=0.8,en-US;q=0.6,en;q=0.4',
          'Cache-Control' : 'max-age=0',
          'Connection': 'keep-alive',
          'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.81 Safari/537.36'}

s = cfscrape.create_scraper()
scraper = cfscrape.create_scraper(sess=s)

In [73]:
all_links = []
for n in range(len(products_collection)):
    
    scraped_html = scraper.get(products_collection[n], headers=header).content
    html = etree.HTML(scraped_html)
        
    # number of pages 
    path = '//*[@class="pagination-bar__link js-queryLink scrollup"]/span//text()'
    pagination = int(html.xpath(path)[-1])
    
    # how many products should be in the section
    path = '//*[@class="results-bar__variants hidden-phone hidden-tablet"]//text()'
    articles_qnt = re.findall(r'\d+(?= Artik)', html.xpath(path)[0])[0]
    articles_qnt = int(articles_qnt)
    
    print('\nSection:', n+1, '|', '#Pages:', pagination, '|',  '#Products:', articles_qnt, '|', products_collection[n])
    
    section_links = []
    for page in range(1, pagination+1):
        
        url = products_collection[n] + '?page={}'
        url = url.format(page)

        scraped_html = scraper.get(url, headers=header).content
        html = etree.HTML(scraped_html)

        page_links = []
        for link in html.xpath('//*[@class="product-wrapper wt_ignore"]//@href'):
            if re.findall(r'\/p\/\d\d\d+', link):
                page_links.append(link)
        section_links.extend(page_links)

        print('Page: ', page, '|', len(page_links), 'links extracted', '|', 'Total links (mined):', len(section_links), '|', 'Total links (website):', articles_qnt)
        if page == pagination and len(section_links) != articles_qnt:
            print('Got less links htan expected for section: ', products_collection[n] )
            break
        elif page == pagination and len(section_links) == articles_qnt:
            print('All links were extracted correctly')
            pass
        
        sleep(5)
    
    all_links.extend(section_links)



Section: 1 | #Pages: 6 | #Products: 368 | https://www.obi.de/gartenbau/terrassendielen/c/1464
Page:  1 | 71 links extracted | Total links (mined): 71 | Total links (website): 368
Page:  2 | 71 links extracted | Total links (mined): 142 | Total links (website): 368
Page:  3 | 71 links extracted | Total links (mined): 213 | Total links (website): 368
Page:  4 | 71 links extracted | Total links (mined): 284 | Total links (website): 368
Page:  5 | 71 links extracted | Total links (mined): 355 | Total links (website): 368
Page:  6 | 13 links extracted | Total links (mined): 368 | Total links (website): 368

Section: 2 | #Pages: 7 | #Products: 440 | https://www.obi.de/laminat-parkett-vinylboeden/laminat/c/514
Page:  1 | 72 links extracted | Total links (mined): 72 | Total links (website): 440
Page:  2 | 72 links extracted | Total links (mined): 144 | Total links (website): 440
Page:  3 | 72 links extracted | Total links (mined): 216 | Total links (website): 440
Page:  4 | 72 links extracted

In [75]:
len(all_links)

3433

In [76]:
# deduplicating links
product_links = []
for i in all_links:
    if i not in product_links:
        product_links.append(i)
len(product_links)

3063

In [82]:
product_links = ['https://www.obi.de' + x for x in product_links]

In [83]:
with open('OBI-Wood-Links.pkl', 'wb') as f:
    pickle.dump(product_links, f)

# Testing options

In [88]:
scraped_html = scraper.get(product_links[1], headers=header).content
html = etree.HTML(scraped_html)

In [118]:
links_options = []
plain_pages = [] 

In [119]:
%%time
# Sorting plain and oprional pages
for i in range(len(product_links)):
    link = product_links[i]
    scraped_html = scraper.get(link, headers=header).content
    html = etree.HTML(scraped_html)

    # Checking multiple options
    raw_options = html.xpath('//*[@class="selectboxes clearfix marg_t5 marg_b20"]//text()')
    if len(raw_options)>0 and link not in links_options:
        links_options.append(link)
    elif len(raw_options)==0 and link not in plain_pages:
        plain_pages.append(link)
    if i%200==0:
        print('#', i)
    
    sleep(1)

# 0
# 200
# 400
# 600
# 800
# 1000
# 1200
# 1400
# 1600
# 1800
# 2000
# 2200
# 2400
# 2600
# 2800
# 3000
CPU times: user 1min 13s, sys: 2.85 s, total: 1min 16s
Wall time: 1h 14min 34s


In [121]:
len(links_options), len(plain_pages), len(links_options)+len(plain_pages)

(943, 2120, 3063)

In [163]:
# Cheking optiopn patterns
opt_df = pd.DataFrame(columns=['Pattern'], index = range(len(links_options)))
for i in range(len(links_options)):
    link = links_options[i]
    scraped_html = scraper.get(link, headers=header).content
    html = etree.HTML(scraped_html)
    sleep(0.3)
    
    # Checking multiple options
    page_options = []
    raw_options = html.xpath('//*[@class="selectboxes clearfix marg_t5 marg_b20"]//text()')
    for opt in raw_options:
        if ':' in opt:
            page_options.append(opt)
    opt_df.loc[i] =  ' '.join(page_options)
    if i%100==0:
        print(i, ' '.join(page_options))
    
    #sleep(1)

0 Farbe: AusfÃ¼hrung:
100 AusfÃ¼hrung:
200 Farbe:
300 Farbe:
400 Farbe: AusfÃ¼hrung:
500 Farbe: AusfÃ¼hrung:
600 Farbe: AusfÃ¼hrung:
700 Farbe:
800 Farbe: AusfÃ¼hrung:
900 Farbe:


In [680]:
with open('OBI-links_options.pkl', 'wb') as f:
    pickle.dump(links_options, f)

In [681]:
with open('OBI-opt_df.pkl', 'wb') as f:
    pickle.dump(opt_df, f)

In [446]:
opt_df['Pattern'].value_counts()

Farbe: Ausführung:    489
Farbe:                382
Ausführung:            72
Name: Pattern, dtype: int64

In [444]:
opt_df['Pattern'].shape

(943,)

In [445]:
opt_df = opt_df.replace('Ã¤', 'ä', regex=True)
opt_df = opt_df.replace('Ã¼', 'ü', regex=True)
opt_df = opt_df.replace('Ã¶', 'ö', regex=True)
opt_df = opt_df.replace('Ã', 'ß', regex=True)

### Getting data inside page

In [545]:
opt_df['Pattern'].sample()

497    Farbe: Ausführung:
Name: Pattern, dtype: object

In [546]:
i=497
browser.get(links_options[i])

### Selecting options
#### Option 1. One Parameter

In [689]:
opt_df[(opt_df['Pattern'] == 'Farbe:') | 
       (opt_df['Pattern'] == 'Ausführung:')].shape

(454, 1)

In [710]:
%%time
option_single_pages_df = pd.DataFrame([])
option_single_pages_df['URL']= np.nan

start=0
indexes = opt_df[(opt_df['Pattern'] == 'Farbe:') | (opt_df['Pattern'] == 'Ausführung:')].index[start:]

for n, i in enumerate(indexes, start=start):
    if n%50==0 or n+1 == len(indexes):
        sleep(randint(1,2))
    
    print('\n')
    print('Optional product:', n+1, links_options[i])
    
    if links_options[i] in option_single_pages_df['URL'].unique():
        continue
    else:
        pass
        
    scraped_html = scraper.get(links_options[i], headers=header).content
    html = etree.HTML(scraped_html)
    
    # Getting links to all selecting options
    opt_links_raw = html.xpath('//*[@class="selectboxes clearfix marg_t5 marg_b20"]//@href') 
    opt_links = []
    for opt_link in opt_links_raw:
        if 'https://' in opt_link and opt_link not in opt_links:
            opt_links.append(opt_link)
    print('Links for product options:', len(opt_links))
   
    
    # Getting data for each option
    
    option_df = pd.DataFrame([])
    for i in range(len(opt_links)):
        sleep(0.5)

        # Getting page data
        url = opt_links[i]
        scraped_html = scraper.get(url, headers=header).content
        html = etree.HTML(scraped_html)

        # Main Table
        soup = BeautifulSoup(scraped_html, 'html.parser')
        tables = soup.findAll("table", { "class" : "c-datalist c-datalist--33" })

        tables_dict = {}
        for count, table in enumerate(tables):
            for element in table.find_all('td'):
                key = element['data-label']+'-T'+ str(count+1)
                tables_dict[key] = element.text
        main_table = pd.DataFrame(tables_dict, index=[0])
        main_table['URL'] = url

        # Title
        path = '//*[@class="h2 overview__heading"]//text()'
        title = html.xpath(path)
        title = '| '.join(title)
        main_table['Title'] = title

        # SKU
        path = '//*[@class="article-number"]//text()'
        sku = html.xpath(path)
        sku = '| '.join(sku)
        main_table['SKU'] = sku

        # Description top
        path = '//*[@class="overview__detail-list normal black"]//text()'
        descr_top = html.xpath(path)
        descr_top = '| '.join(descr_top)

        # Description bottom
        path = '//*[@data-ui-name="ads.description-text.product-details.p"]//text()'
        bottom_descr = html.xpath(path)
        bottom_descr = '| '.join(bottom_descr)

        main_table['Description'] = descr_top + ' | ' + bottom_descr

        # Price
        path = '//*[@class="span12 span-mobile12"]//text()'
        price = html.xpath(path)
        price = '| '.join(price)
        main_table['Price Raw'] = price

        # Delivery
        path = '//*[@class="button-to-card__txt--table-cell"]//text()'
        delivery = html.xpath(path)
        delivery = '| '.join(delivery)
        main_table['Delivery'] = delivery

        option_df = pd.concat([option_df, main_table], ignore_index=True)
        
        if ((i+1)%10==0 or i+1==len(all_options)) and i>0:
            print('Processed option:', i+1, '|', 'Table length:', option_df.shape[0])
            sleep(0.5)
        
    option_single_pages_df = pd.concat([option_single_pages_df, option_df], ignore_index=True)
    
    print('Products Dataframe Length:', option_single_pages_df.shape[0])
    if option_single_pages_df.shape[0] != option_single_pages_df.drop_duplicates().shape[0]:
        print('Products Dataframe Length NO DUPL:', option_single_pages_df.drop_duplicates().shape[0])



Optional product: 1 https://www.obi.de/terrassendielen/unterkonstruktion-douglasie-4-5-cm-x-7-cm-x-200-cm/p/7239882
Links for product options: 2
Products Dataframe Length: 2


Optional product: 2 https://www.obi.de/terrassendielen/terrassendiele-douglasie-premium-teil-geriffelt-glatt-2-1-cm-x-12-cm-x-200-cm/p/2954774
Links for product options: 4
Processed option: 4 | Table length: 4
Products Dataframe Length: 6


Optional product: 3 https://www.obi.de/terrassendielen/terrassendiele-midsommar-vorvergraut-14-cm-x-200-cm/p/2754612
Links for product options: 1
Products Dataframe Length: 7


Optional product: 4 https://www.obi.de/holzfliesen/holzfliese-geriffelt-gruen-druckimpraegniert-50-cm-x-3-2-cm-x-50-cm/p/2463560
Links for product options: 2
Products Dataframe Length: 9


Optional product: 5 https://www.obi.de/terrassendielen/terrassendiele-laerche-premium-teil-geriffelt-glatt-2-8-cm-x-14-5-cm-x-200-cm/p/2955052
Links for product options: 2
Products Dataframe Length: 11


Optional pr

Links for product options: 2
Products Dataframe Length: 90


Optional product: 48 https://www.obi.de/terrassendielen/osmo-bpc-startklammer-multi-deck-20-mm-10er-pack/p/3679941
Links for product options: 2
Products Dataframe Length: 92


Optional product: 49 https://www.obi.de/terrassendielen/osmo-befestigungsklammer-multi-deck-uk-28-x-50-10er-pack/p/3680048
Links for product options: 2
Products Dataframe Length: 94


Optional product: 50 https://www.obi.de/terrassendielen/osmo-bpc-befestigungsclip-multi-deck-25-mm-25er-pack/p/3680303
Links for product options: 2
Products Dataframe Length: 96


Optional product: 51 https://www.obi.de/terrassendielen/osmo-terrassendiele-sibirische-laerche-glatt-2-7-cm-x-14-3-cm-x-300-cm/p/4246856
Links for product options: 1
Products Dataframe Length: 97


Optional product: 52 https://www.obi.de/terrassendielen/unterkonstruktion-hartholz-4-5-cm-x-7-cm-x-213-cm/p/6102164
Links for product options: 2
Products Dataframe Length: 99


Optional product: 53 htt

Products Dataframe Length: 216
Products Dataframe Length NO DUPL: 211


Optional product: 95 https://www.obi.de/laminat/schoener-wohnen-laminatboden-klassik-trendeiche-weiss/p/4180444
Links for product options: 3
Products Dataframe Length: 219
Products Dataframe Length NO DUPL: 214


Optional product: 96 https://www.obi.de/laminat/egger-home-laminatboden-classic-dimas-wood-bunt/p/6688584


Optional product: 97 https://www.obi.de/laminat/laminatboden-excellent-celtic-oak-eiche-altholzstruktur-hell/p/5145842


Optional product: 98 https://www.obi.de/laminat/schoener-wohnen-laminatboden-klassik-herbsteiche-grau/p/4180485


Optional product: 99 https://www.obi.de/laminat/laminatboden-comfort-pinie-mokka-dunkel/p/5145081


Optional product: 100 https://www.obi.de/laminat/megafloor-laminatboden-western-eiche-natur/p/6687032


Optional product: 101 https://www.obi.de/laminat/laminatboden-comfort-eiche-antik-grau-7-mm/p/3168770


Optional product: 102 https://www.obi.de/laminat/parador-laminat

Links for product options: 3
Products Dataframe Length: 278
Products Dataframe Length NO DUPL: 273


Optional product: 163 https://www.obi.de/laminat/living-by-haro-laminatboden-sonderedition-nussbaum/p/1897578
Links for product options: 1
Products Dataframe Length: 279
Products Dataframe Length NO DUPL: 274


Optional product: 164 https://www.obi.de/laminat/parador-laminatboden-classic-1050-walnuss-gekaelkt-schiffsboden/p/6546147


Optional product: 165 https://www.obi.de/laminat/parador-laminatboden-trendtime-6-eiche-castell-gekaelkt/p/6547434


Optional product: 166 https://www.obi.de/laminat/parador-laminatboden-trendtime-1-esche-gealtert-natur-rustikalstruktur/p/3601978


Optional product: 167 https://www.obi.de/laminat/parador-laminatboden-classic-1050-eiche-lichtgrau-landhausdiele-v-fuge/p/6546493


Optional product: 168 https://www.obi.de/laminat/parador-laminatboden-trendtime-6-eiche-askada-weiss-gekaelkt-schlossdiele/p/6547350


Optional product: 169 https://www.obi.de/lamina

Links for product options: 5
Processed option: 4 | Table length: 4
Products Dataframe Length: 290
Products Dataframe Length NO DUPL: 285


Optional product: 230 https://www.obi.de/konstruktionsvollholz-brettschichtholz/konstruktionsvollholz-nsi-fichte-tanne-60-mm-x-100-mm-x-6000-mm/p/2002442
Links for product options: 7
Processed option: 4 | Table length: 4
Products Dataframe Length: 297
Products Dataframe Length NO DUPL: 292


Optional product: 231 https://www.obi.de/konstruktionsvollholz-brettschichtholz/konstruktionsholz-laerche-glatt-200-cm-x-4-5-cm-x-2-8-cm/p/6055099
Links for product options: 2
Products Dataframe Length: 299
Products Dataframe Length NO DUPL: 294


Optional product: 232 https://www.obi.de/konstruktionsvollholz-brettschichtholz/konstruktionsbrett-kreativo-douglasie-200-cm-x-9-5-cm-x-1-9-cm/p/2753846
Links for product options: 2
Products Dataframe Length: 301
Products Dataframe Length NO DUPL: 296


Optional product: 233 https://www.obi.de/hobelware/mehrzwecktafel-

Links for product options: 15
Processed option: 4 | Table length: 4
Processed option: 10 | Table length: 10
Products Dataframe Length: 437
Products Dataframe Length NO DUPL: 432


Optional product: 282 https://www.obi.de/arbeitsplatten/kantenumleimer-65-cm-x-4-4-cm-mosaik-braun-beige-ms-234-2-stueck/p/5701511


Optional product: 283 https://www.obi.de/arbeitsplatten/kantenumleimer-65-cm-x-4-4-cm-buche-gedaempft-bu-95-2-stueck/p/1414226


Optional product: 284 https://www.obi.de/arbeitsplatten/innenecke-plus-kaviar/p/5458302


Optional product: 285 https://www.obi.de/arbeitsplatten/kantenumleimer-65-cm-x-4-4-cm-weiss-grau-s-210-2-stueck/p/1794239


Optional product: 286 https://www.obi.de/arbeitsplatten/kantenumleimer-65-cm-x-4-4-cm-granit-anthrazit-gt-117-2-stueck/p/1794247


Optional product: 287 https://www.obi.de/arbeitsplatten/kantenumleimer-65-cm-x-4-4-cm-sandy-dunkel-grau-s-414-2-stueck/p/1794254


Optional product: 288 https://www.obi.de/arbeitsplatten/kantenumleimer-65-cm-x-4-4

Links for product options: 4
Processed option: 4 | Table length: 4
Products Dataframe Length: 446
Products Dataframe Length NO DUPL: 441


Optional product: 356 https://www.obi.de/arbeitsplatten/kantenumleimer-65-cm-x-4-4-cm-nussbaum-nu-795-2-stueck/p/1673789


Optional product: 357 https://www.obi.de/arbeitsplatten/innenecke-compact-hellbeige/p/6694749
Links for product options: 4
Processed option: 4 | Table length: 4
Products Dataframe Length: 450
Products Dataframe Length NO DUPL: 445


Optional product: 358 https://www.obi.de/arbeitsplatten/innenecke-compact-dunkelbraun/p/6694764


Optional product: 359 https://www.obi.de/arbeitsplatten/innenecke-compact-lichtgrau/p/6694806


Optional product: 360 https://www.obi.de/arbeitsplatten/innenecke-compact-anthrazit/p/6694814


Optional product: 361 https://www.obi.de/arbeitsplatten/aussenecke-compact-zimt/p/6697411


Optional product: 362 https://www.obi.de/arbeitsplatten/endkappen-compact-hellbeige/p/6697569


Optional product: 363 https

Links for product options: 3
Products Dataframe Length: 536
Products Dataframe Length NO DUPL: 531


Optional product: 399 https://www.obi.de/regalboeden-moebelbauplatten/obi-kubus-set-roanne-weiss-3-teilig/p/3875325
Links for product options: 2
Products Dataframe Length: 538
Products Dataframe Length NO DUPL: 533


Optional product: 400 https://www.obi.de/regalboeden-moebelbauplatten/obi-buecherstuetze-weiss-12-cm-x-13-cm-4-stueck/p/6314199
Links for product options: 2
Products Dataframe Length: 540
Products Dataframe Length NO DUPL: 535


Optional product: 401 https://www.obi.de/regalboeden-moebelbauplatten/obi-eck-wandregal-birke-29-5-x-29-5-cm/p/2809168


Optional product: 402 https://www.obi.de/regalboeden-moebelbauplatten/obi-glasboden-villar-gerundete-ecken-klar-0-6-cm-x-60-cm-x-15-cm/p/2809317
Links for product options: 2
Products Dataframe Length: 542
Products Dataframe Length NO DUPL: 537


Optional product: 403 https://www.obi.de/regalboeden-moebelbauplatten/befestigungsring

Links for product options: 2
Products Dataframe Length: 582
Products Dataframe Length NO DUPL: 577


Optional product: 448 https://www.obi.de/regalboeden-moebelbauplatten/obi-fix-set-mattsilber/p/3872884


Optional product: 449 https://www.obi.de/regalboeden-moebelbauplatten/einloch-buecherbuegel-silber-14-3-cm-x-11-8-cm/p/7096175
Links for product options: 1
Products Dataframe Length: 583
Products Dataframe Length NO DUPL: 578


Optional product: 450 https://www.obi.de/regalboeden-moebelbauplatten/obi-befestigungsclip-select-schwarz/p/3846227
Links for product options: 4
Processed option: 4 | Table length: 4
Products Dataframe Length: 587
Products Dataframe Length NO DUPL: 582


Optional product: 451 https://www.obi.de/regalboeden-moebelbauplatten/obi-befestigungsclip-select-silber-matt/p/3871654


Optional product: 452 https://www.obi.de/regalboeden-moebelbauplatten/obi-befestigungsclip-select-silber/p/3871662


Optional product: 453 https://www.obi.de/regalboeden-moebelbauplatten/ob

In [714]:
missed_links = [x for x in [links_options[i] for i in indexes] if x not in option_single_pages_df['URL'].unique()]
len(missed_links)

3

In [717]:
plain_pages.extend(missed_links)

#### Option 2. Two Parameters

In [674]:
%%time
#option_pages_df = pd.DataFrame([])
#option_pages_df['URL']= np.nan

start=0
for n, i in enumerate(opt_df[opt_df['Pattern'] == 'Farbe: Ausführung:'].index[start:], start=start):
    if n%50==0 or n+1 == opt_df[opt_df['Pattern'] == 'Farbe: Ausführung:'].shape[0]:
        sleep(randint(1,2))
    
    print('\n')
    print('Optional product:', n+1, links_options[i])
    
    if links_options[i] in option_pages_df['URL'].unique():
        continue
    else:
        pass
        
    scraped_html = scraper.get(links_options[i], headers=header).content
    html = etree.HTML(scraped_html)
    
    # Getting links to all selecting options
    opt_links_raw = html.xpath('//*[@data-ui-name="ads.variants.color.enabled"]//@href') 
    opt_links = []
    for opt_link in opt_links_raw:
        if 'https://' in opt_link and opt_link not in opt_links:
            opt_links.append(opt_link)
    
    all_options = []
    all_options.extend(opt_links)
    for i, opt_link in enumerate(opt_links):
        if (i+1)%10==0 or i+1 == len(opt_links):
            print(i+1, 'out of', len(opt_links), 'main options', '|', 'Links for product options:', len(all_options))

        scraped_html = scraper.get(opt_link, headers=header).content
        html = etree.HTML(scraped_html)
        child_links = html.xpath('//*[@class="selectbox child3"]//@href')
        
        sleep(1)
        for child_link in child_links:
            if child_link not in all_options and 'https://' in child_link:
                all_options.append(child_link)
    
    # Getting data for each option
    
    option_df = pd.DataFrame([])
    for i in range(len(all_options)):
        sleep(0.5)

        # Getting page data
        url = all_options[i]
        scraped_html = scraper.get(url, headers=header).content
        html = etree.HTML(scraped_html)

        # Main Table
        soup = BeautifulSoup(scraped_html, 'html.parser')
        tables = soup.findAll("table", { "class" : "c-datalist c-datalist--33" })

        tables_dict = {}
        for count, table in enumerate(tables):
            for element in table.find_all('td'):
                key = element['data-label']+'-T'+ str(count+1)
                tables_dict[key] = element.text
        main_table = pd.DataFrame(tables_dict, index=[0])
        main_table['URL'] = url

        # Title
        path = '//*[@class="h2 overview__heading"]//text()'
        title = html.xpath(path)
        title = '| '.join(title)
        main_table['Title'] = title

        # SKU
        path = '//*[@class="article-number"]//text()'
        sku = html.xpath(path)
        sku = '| '.join(sku)
        main_table['SKU'] = sku

        # Description top
        path = '//*[@class="overview__detail-list normal black"]//text()'
        descr_top = html.xpath(path)
        descr_top = '| '.join(descr_top)

        # Description bottom
        path = '//*[@data-ui-name="ads.description-text.product-details.p"]//text()'
        bottom_descr = html.xpath(path)
        bottom_descr = '| '.join(bottom_descr)

        main_table['Description'] = descr_top + ' | ' + bottom_descr

        # Price
        path = '//*[@class="span12 span-mobile12"]//text()'
        price = html.xpath(path)
        price = '| '.join(price)
        main_table['Price Raw'] = price

        # Delivery
        path = '//*[@class="button-to-card__txt--table-cell"]//text()'
        delivery = html.xpath(path)
        delivery = '| '.join(delivery)
        main_table['Delivery'] = delivery

        option_df = pd.concat([option_df, main_table], ignore_index=True)
        
        if ((i+1)%10==0 or i+1==len(all_options)) and i>0:
            print('Processed option:', i+1, '|', 'Table length:', option_df.shape[0])
            sleep(0.5)
        
    option_pages_df = pd.concat([option_pages_df, option_df], ignore_index=True)
    
    print('Products Dataframe Length:', option_pages_df.shape[0])
    if option_pages_df.shape[0] != option_pages_df.drop_duplicates().shape[0]:
        print('Products Dataframe Length NO DUPL:', option_pages_df.drop_duplicates().shape[0])



Optional product: 1 https://www.obi.de/terrassendielen/kovalex-alu-seitenabschluss-eloxiert-28-mm-x-27-mm-x-2500-mm/p/8103616


Optional product: 2 https://www.obi.de/terrassendielen/osmo-bpc-terrassendiele-multi-deck-co-extrusion-gray-2-x-14-5-x-300-cm/p/3679099


Optional product: 3 https://www.obi.de/terrassendielen/osmo-bpc-terrassendiele-multi-deck-co-extrusion-ipe-2-1-x-14-5-x-300-cm/p/3679214


Optional product: 4 https://www.obi.de/terrassendielen/osmo-bpc-terrassendiele-multi-deck-co-extrusion-ipe-2-x-14-5-x-300-cm/p/3678984


Optional product: 5 https://www.obi.de/terrassendielen/osmo-bpc-terrassendiele-multi-deck-co-extrusion-walnuss-2-x-14-5-x-300-cm/p/3679024


Optional product: 6 https://www.obi.de/terrassendielen/osmo-bpc-terrassendiele-multi-deck-co-extrusion-walnuss-2-1-x-14-5-x-300-cm/p/3679339


Optional product: 7 https://www.obi.de/terrassendielen/osmo-bpc-unterkonstruktion-multi-deck-braun-laenge-3-m-staerke-2-8-mm/p/3679388


Optional product: 8 https://www.obi

10 out of 18 main options | Links for product options: 27
18 out of 18 main options | Links for product options: 32
Processed option: 10 | Table length: 10
Processed option: 20 | Table length: 20
Processed option: 30 | Table length: 30
Processed option: 33 | Table length: 33


Optional product: 35 https://www.obi.de/arbeitsplatten/wandanschlussleiste-plus-59-x-3-cm-buche-parkett-bu-37-/p/2107266
10 out of 18 main options | Links for product options: 27
18 out of 18 main options | Links for product options: 32
Processed option: 10 | Table length: 10
Processed option: 20 | Table length: 20
Processed option: 30 | Table length: 30
Processed option: 33 | Table length: 33


Optional product: 36 https://www.obi.de/arbeitsplatten/wandanschlussleiste-plus-59-x-3-cm-tumalin-grau-beige-h-272-/p/6438824
10 out of 18 main options | Links for product options: 27
18 out of 18 main options | Links for product options: 32
Processed option: 10 | Table length: 10
Processed option: 20 | Table length: 20
P

Processed option: 2 | Table length: 2


Optional product: 58 https://www.obi.de/arbeitsplatten/aussenecke-system-dnp-holz-holz-/p/2498871
2 out of 2 main options | Links for product options: 2
Processed option: 2 | Table length: 2


Optional product: 59 https://www.obi.de/arbeitsplatten/aussenecke-system-dnp-kiesel-kiesel-/p/2498905
2 out of 2 main options | Links for product options: 2
Processed option: 2 | Table length: 2


Optional product: 60 https://www.obi.de/regalboeden-moebelbauplatten/regalboden-weiss-60-cm-x-20-cm-x-1-6-cm/p/9067190
7 out of 7 main options | Links for product options: 61
Processed option: 10 | Table length: 10
Processed option: 20 | Table length: 20
Processed option: 30 | Table length: 30
Processed option: 40 | Table length: 40
Processed option: 50 | Table length: 50
Processed option: 60 | Table length: 60
Processed option: 70 | Table length: 70
Processed option: 80 | Table length: 80
Processed option: 83 | Table length: 83


Optional product: 61 https://www.

1 out of 1 main options | Links for product options: 1
Processed option: 2 | Table length: 2


Optional product: 85 https://www.obi.de/regalboeden-moebelbauplatten/tischplatte-fichte-120-cm-x-80-cm-x-2-8-cm/p/4299954
3 out of 3 main options | Links for product options: 5
Processed option: 6 | Table length: 6


Optional product: 86 https://www.obi.de/regalboeden-moebelbauplatten/stahlrohr-zum-moebelbau-30-cm-2-7-cm-schwarz/p/4028627
2 out of 2 main options | Links for product options: 7
Processed option: 10 | Table length: 10
Processed option: 13 | Table length: 13


Optional product: 87 https://www.obi.de/regalboeden-moebelbauplatten/einloch-winkeltraeger-weiss-18-cm-2-stueck/p/2664761
3 out of 3 main options | Links for product options: 9
Processed option: 10 | Table length: 10
Processed option: 12 | Table length: 12


Optional product: 88 https://www.obi.de/regalboeden-moebelbauplatten/regalboden-buche-holznachbildung-80-cm-x-20-cm-x-1-6-cm/p/1749761
8 out of 8 main options | Links f

7 out of 7 main options | Links for product options: 61
Processed option: 10 | Table length: 10
Processed option: 20 | Table length: 20
Processed option: 30 | Table length: 30
Processed option: 40 | Table length: 40
Processed option: 50 | Table length: 50
Processed option: 60 | Table length: 60
Processed option: 70 | Table length: 70
Processed option: 80 | Table length: 80
Processed option: 83 | Table length: 83


Optional product: 115 https://www.obi.de/regalboeden-moebelbauplatten/regalboden-ahorn-holznachbildung-80-cm-x-20-cm-x-1-6-cm/p/4771556
8 out of 8 main options | Links for product options: 62
Processed option: 10 | Table length: 10
Processed option: 20 | Table length: 20
Processed option: 30 | Table length: 30
Processed option: 40 | Table length: 40
Processed option: 50 | Table length: 50
Processed option: 60 | Table length: 60
Processed option: 70 | Table length: 70
Processed option: 80 | Table length: 80
Processed option: 84 | Table length: 84


Optional product: 116 https:

Processed option: 6 | Table length: 6


Optional product: 147 https://www.obi.de/regalboeden-moebelbauplatten/obi-alukonsole-bea-alu-silber-100-mm-x-150-mm/p/8650640
2 out of 2 main options | Links for product options: 5
Processed option: 8 | Table length: 8


Optional product: 148 https://www.obi.de/regalboeden-moebelbauplatten/obi-wandregal-weiss-79-5-cm-x-19-5-cm/p/2811032
2 out of 2 main options | Links for product options: 3
Processed option: 3 | Table length: 3


Optional product: 149 https://www.obi.de/regalboeden-moebelbauplatten/obi-wandregal-birke-59-5-cm-x-19-5-cm/p/2809176
2 out of 2 main options | Links for product options: 3
Processed option: 3 | Table length: 3


Optional product: 150 https://www.obi.de/regalboeden-moebelbauplatten/obi-multiplex-konsole-buche-150-mm-x-100-mm/p/7156565
1 out of 1 main options | Links for product options: 1
Processed option: 4 | Table length: 4


Optional product: 151 https://www.obi.de/regalboeden-moebelbauplatten/obi-wandregal-valbonne-c



Optional product: 184 https://www.obi.de/regalboeden-moebelbauplatten/konfektionstraeger-silber-25-cm/p/7100175
2 out of 2 main options | Links for product options: 3
Processed option: 4 | Table length: 4


Optional product: 185 https://www.obi.de/regalboeden-moebelbauplatten/obi-regalboden-eckig-buche-holznachbildung-1-8-cm-x-60-cm-x-20-cm/p/2811453
1 out of 1 main options | Links for product options: 1
Processed option: 2 | Table length: 2


Optional product: 186 https://www.obi.de/regalboeden-moebelbauplatten/obi-wandregal-monteux-congo-59-5-cm-x-19-5-cm-x-1-8-cm/p/5562566
2 out of 2 main options | Links for product options: 3
Processed option: 4 | Table length: 4


Optional product: 187 https://www.obi.de/regalboeden-moebelbauplatten/obi-wandregal-monteux-petrol-59-5-cm-x-19-5-cm-x-1-8-cm/p/5562574
2 out of 2 main options | Links for product options: 3
Processed option: 4 | Table length: 4
CPU times: user 8min 20s, sys: 8.68 s, total: 8min 28s
Wall time: 1h 12min 44s


In [684]:
idx = opt_df[opt_df['Pattern'] == 'Farbe: Ausführung:'].index.to_list()
missed_links = [x for x in [links_options[i] for i in idx] if x not in option_pages_df['URL'].unique()]
len(missed_links)

11

In [718]:
plain_pages.extend(missed_links)

In [687]:
option_pages_df.shape

(16240, 34)

In [688]:
option_pages_df = option_pages_df.drop_duplicates()
option_pages_df.shape

(2084, 34)

## Getting data for the pages without selecting options

### Looping through pages

In [404]:
plain_pages_df = pd.DataFrame([])
for i in range(len(plain_pages)):
    
    if i%100==0:
        print('#', i, '|', 'Table length:', plain_pages_df.shape[0])
        sleep(randint(1,4))
    sleep(1)
    
    # Getting page data
    scraped_html = scraper.get(plain_pages[i], headers=header).content
    html = etree.HTML(scraped_html)
    
    # Main Table
    soup = BeautifulSoup(scraped_html, 'html.parser')
    tables = soup.findAll("table", { "class" : "c-datalist c-datalist--33" })
    
    tables_dict = {}
    for i, table in enumerate(tables):
        for element in table.find_all('td'):
            key = element['data-label']+'-T'+ str(i+1)
            tables_dict[key] = element.text
    main_table = pd.DataFrame(tables_dict, index=[0])
    main_table['URL'] = plain_pages[i]
    
    # Title
    path = '//*[@class="h2 overview__heading"]//text()'
    title = html.xpath(path)
    title = '| '.join(title)
    main_table['Title'] = title
    
    # SKU
    path = '//*[@class="article-number"]//text()'
    sku = html.xpath(path)
    sku = '| '.join(sku)
    main_table['SKU'] = sku
    
    # Description top
    path = '//*[@class="overview__detail-list normal black"]//text()'
    descr_top = html.xpath(path)
    descr_top = '| '.join(descr_top)
    
    # Description bottom
    path = '//*[@data-ui-name="ads.description-text.product-details.p"]//text()'
    bottom_descr = html.xpath(path)
    bottom_descr = '| '.join(bottom_descr)
    
    main_table['Description'] = descr_top + ' | ' + bottom_descr
    
    # Price
    path = '//*[@class="span12 span-mobile12"]//text()'
    price = html.xpath(path)
    price = '| '.join(price)
    main_table['Price Raw'] = price
    
    # Delivery
    path = '//*[@class="button-to-card__txt--table-cell"]//text()' #button-to-card row-fluid disabled
    delivery = html.xpath(path)
    delivery = '| '.join(delivery)
    main_table['Delivery'] = delivery
    
    plain_pages_df = pd.concat([plain_pages_df, main_table], ignore_index=True)

# 0 | Table length: 0
# 50 | Table length: 50
# 100 | Table length: 100
# 150 | Table length: 150
# 200 | Table length: 200
# 250 | Table length: 250
# 300 | Table length: 300
# 350 | Table length: 350
# 400 | Table length: 400
# 450 | Table length: 450
# 500 | Table length: 500
# 550 | Table length: 550
# 600 | Table length: 600
# 650 | Table length: 650
# 700 | Table length: 700
# 750 | Table length: 750
# 800 | Table length: 800
# 850 | Table length: 850
# 900 | Table length: 900
# 950 | Table length: 950
# 1000 | Table length: 1000
# 1050 | Table length: 1050
# 1100 | Table length: 1100
# 1150 | Table length: 1150
# 1200 | Table length: 1200
# 1250 | Table length: 1250
# 1300 | Table length: 1300
# 1350 | Table length: 1350
# 1400 | Table length: 1400
# 1450 | Table length: 1450
# 1500 | Table length: 1500
# 1550 | Table length: 1550
# 1600 | Table length: 1600
# 1650 | Table length: 1650
# 1700 | Table length: 1700
# 1750 | Table length: 1750
# 1800 | Table length: 1800
# 1850 | T

In [410]:
plain_pages_df.shape

(2120, 37)

In [590]:
with open('OBI-Plain-Pages.pkl', 'wb') as f:
    pickle.dump(plain_pages_df, f)

In [428]:
for i in range(len(plain_pages)):
    plain_pages_df['URL'].loc[i] = plain_pages[i]

# Processing data

In [719]:
products = pd.concat([option_single_pages_df, option_pages_df, plain_pages_df], ignore_index=True)
products.shape

(4791, 37)

#### Deduplication

In [721]:
products = products[~products.duplicated(keep='last')]
products.shape

(4786, 37)

#### Improving text

In [725]:
products = products.replace('Ã¤', 'ä', regex=True)
products = products.replace('Ã¼', 'ü', regex=True)
products = products.replace('Ã¶', 'ö', regex=True)
products = products.replace('Ã', 'ß', regex=True)
products = products.replace('â¬', '€', regex=True)

In [1758]:
with open('OBI-Wood-Products-Raw.pkl', 'wb') as f:
    pickle.dump(products, f)

#### Price/Delivery

In [1624]:
test = products.copy()

In [1625]:
test['Price'] = test['Price Raw'].str.replace(r'\| +\| +','| ').str.replace(r'^\D+','')
test['Price'] = test['Price'].str.extract(r'(.+?(?=\|))')[0]
test['Price'] = test['Price'].str.replace(r'\,','\.').str.replace(r'[^\.\d]|\.00','').astype(float)

#### Delivery and Order Online

In [1626]:
test['Delivery Status'] = test['Delivery'].str.extract(r'(ca\. \d+\-\d+ .+?(?=\|))')[0]
pat = test['Delivery'].str.extract(r'(sofort lieferbar .+?(?=\|))')[0]
test['Delivery Status'] = test['Delivery Status'].fillna(pat)
test['Delivery Yes/No'] = test['Delivery Status'].str.replace(r'.+','Yes').fillna('No')

#### SKU

In [1627]:
test['SKU'] = test['SKU'].str.replace(r'\D','')
test['SKU'].sample(2)

4      2954790
513    7324304
Name: SKU, dtype: object

#### Drop missed columns

In [1628]:
missed_cols = pd.DataFrame((test.isna().sum()/test.shape[0]*100), columns=['Missed'])
cols_del = missed_cols[missed_cols['Missed']>96].index.to_list()
len(cols_del)

6

In [1629]:
test = test.drop(columns=cols_del)
test.shape

(4786, 34)

### Dimentions

#### Unifying to mm

In [1630]:
test['Title-Dims'] = test['Title'].str.replace(r'^.+?(?=\d)','')

In [1631]:
for i in test.index:
    if len(re.findall(r'(?<=\d) m|(?<=\d)m|(?<=\d) cm|(?<=\d)cm', test['Title-Dims'].loc[i]))==0:
        test['Title-Dims'].loc[i] = np.nan

In [1632]:
test['Title-Dims'] = test['Title-Dims'].str.replace(r'[^cmx\d\,\.]','').str.replace(r'^\D+?(?=\d)','')
test['Title-Dims'] = test['Title-Dims'].str.replace(r'cm[^x]','cm').str.replace(r'mm[^x]','mm')
test['Title-Dims'] = test['Title-Dims'].str.replace(r'cm[^x]','cm').str.replace(r'mm[^x]','mm')

In [1633]:
test['Header-Dims-Mult'] = test['Title-Dims']
for i in test.index:
    if 'cm' in test['Header-Dims-Mult'].fillna('').loc[i]:
        test['Header-Dims-Mult'].loc[i] = 10

In [1634]:
for i in test.index:
    if 'mm' in str(test['Header-Dims-Mult'].fillna('').loc[i]):
        test['Header-Dims-Mult'].loc[i] = 1

In [1635]:
header_dims = test['Title-Dims'].str.split('x', expand=True)
header_dims.isna().sum()/header_dims.shape[0]*100

0    29.105725
1    33.765148
2    55.348934
3    99.749269
dtype: float64

In [1636]:
header_dims = header_dims.drop(columns=3).rename(columns={0:'H-Dim1', 1:'H-Dim2', 2:'H-Dim3'})
for col in ['H-Dim1', 'H-Dim2', 'H-Dim3']:
    header_dims[col] = header_dims[col].str.replace(r'\,','.').str.replace(r'[^\.\d]','')
    header_dims[col] = header_dims[col].apply(lambda x: np.nan if x=='' or x ==' '  else x)
   # header_dims[col] = header_dims[col].astype(float)
    #

In [1637]:
test = pd.concat([test, header_dims], axis=1)

### Checking missed dims

In [1638]:
dims_cols = ['Stärke-T1', 'Breite-T1', 'Länge-T1','Höhe-T2','Breite-T2','Tiefe-T2','H-Dim1','H-Dim2','H-Dim3']
test[test['Stärke-T1'].isna()][dims_cols].sample(2)

,Stärke-T1,Breite-T1,Länge-T1,Höhe-T2,Breite-T2,Tiefe-T2,H-Dim1,H-Dim2,H-Dim3
3668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2096,NaN,NaN,NaN,"3,0 cm","59,0 cm","2,0 cm",59,3396,None


### Getting Multipliers

In [1639]:
cols = ['Stärke-T1','Breite-T1','Länge-T1','Höhe-T2','Breite-T2','Tiefe-T2']
for col in cols:
    mult_col = col + '-Mult'
    mm_col = col + '-mm'
    test[mult_col] = 1
    
    test[col] = test[col].astype(str)
    for i in test.index:
        if 'cm' in test[col].loc[i]:
            test[mult_col].loc[i] = 10
            continue

        elif 'mm' in test[col].loc[i]:
            test[mult_col].loc[i] = 1
            continue

        elif 'm' in test[col].loc[i]:
            test[mult_col].loc[i] = 1000
    test[mult_col] = test[mult_col].astype(float)
    test[mm_col] = test[col].str.replace(r'\,','.').str.replace(r'[^\.\d]','')
    test[mm_col] = test[mm_col].apply(lambda x: np.nan if x =='' or x==' ' else x).astype(float)
    
    test[mm_col] = test[mm_col]*test[mult_col]

In [1640]:
new_dims_cols = ['Stärke-T1','Stärke-T1-mm', 
                 'Breite-T1', 'Breite-T1-mm',
                 'Länge-T1','Länge-T1-mm',
                 'Höhe-T2','Höhe-T2-mm',
                 'Breite-T2','Breite-T2-mm',
                 'Tiefe-T2','Tiefe-T2-mm',
                 'H-Dim1','H-Dim2','H-Dim3','Header-Dims-Mult']
test[new_dims_cols].sample(1)

,Stärke-T1,Stärke-T1-mm,Breite-T1,Breite-T1-mm,Länge-T1,Länge-T1-mm,Höhe-T2,Höhe-T2-mm,Breite-T2,Breite-T2-mm,Tiefe-T2,Tiefe-T2-mm,H-Dim1,H-Dim2,H-Dim3,Header-Dims-Mult
1401,"2,7 cm",27.0,"14,5 cm",145.0,3 m,3000.0,"3,00 m",3000.0,"14,5 cm",145.0,"2,7 cm",27.0,27,145,3000,1


In [1641]:
dims_test = test[new_dims_cols].copy()

In [1642]:
cols=['Stärke-T1-mm','Breite-T1-mm','Länge-T1-mm']
sec_cols = ['Tiefe-T2-mm','Breite-T2-mm','Höhe-T2-mm']
for col in cols:
    vals=[]
    for sec_col in sec_cols: 
        val = dims_test[(~dims_test[col].isna()) &
                          (dims_test[col] - dims_test[sec_col]) == 0].shape[0]
        vals.append(val)
    print(vals)
    print(vals.index(max(vals)))
    print(col, sec_cols[vals.index(max(vals))])

[3459, 2056, 3074]
0
Stärke-T1-mm Tiefe-T2-mm
[2415, 4157, 2054]
1
Breite-T1-mm Breite-T2-mm
[2806, 2413, 3018]
2
Länge-T1-mm Höhe-T2-mm


In [1643]:
test['Stärke-T1-mm'] = test['Stärke-T1-mm'].fillna(test['Tiefe-T2-mm'])
test['Länge-T1-mm'] = test['Länge-T1-mm'].fillna(test['Höhe-T2-mm'])
test['Breite-T1-mm'] = test['Breite-T1-mm'].fillna(test['Breite-T2-mm'])

In [1644]:
test[(test['Stärke-T1-mm'].isna()) & 
     (test['Länge-T1-mm'].isna()) & 
     (test['Breite-T1-mm'].isna()) &
     (~test['H-Dim1'].isna())]['Title'].shape[0]/test.shape[0]*100

1.6506477225240284

### Missed Dimentions

In [1745]:
test[(test['Stärke-T1-mm'].isna()) & 
     (test['Länge-T1-mm'].isna()) & 
     (test['Breite-T1-mm'].isna())]['Title'].shape[0]/test.shape[0]*100

15.98279285241562

In [1753]:
test[(test['Stärke-T1-mm'].isna()) & 
     (test['Länge-T1-mm'].isna()) & 
     (test['Breite-T1-mm'].isna())].shape#['Title'].value_counts()

(483, 63)

# *

# Combine all columns to mine text form

In [1645]:
test['Combined'] = ''
for col in test.columns[1:]:
    test['Combined'] = test['Combined'] + '; ' + test[col].astype(str)

In [1646]:
test['Combined'] = test['Combined'].str.replace(r'\; nan','').str.replace(r'^; ','')

### Surface Quality

In [1647]:
quality = ['A-Sortierung', 'B-Sortierung', 'C-Sortierung', 'A/B', 'B/C','Kl. A', '1.Wahl Sortierung', 'Altholz']

In [1648]:
test['Combined Quality'] = ''
for i in test.index:
    test['Combined Quality'].loc[i] = []
    for qual in quality:
        pat = r' {} |^{} | {}$| {}\;'.format(qual.lower(), qual.lower(), qual.lower(), qual.lower())
        if len(re.findall(pat, test['Combined'].loc[i].lower()))>0 and qual.lower() not in ', '.join(test['Combined Quality'].loc[i]):
            test['Combined Quality'].loc[i].append(qual)
            
    try:
        test['Combined Quality'].loc[i] = ', '.join(test['Combined Quality'].loc[i])
    except:
        pass
test['Combined Quality'] =  test['Combined Quality'].apply(lambda x: np.nan if x==' ' or x=='' else x )
test['Combined Quality'].value_counts()

B-Sortierung    17
Altholz         16
C-Sortierung     6
A-Sortierung     2
Name: Combined Quality, dtype: int64

### Sorting Class

In [1649]:
sort_classes = ['S7', 'S10', 'S13', 'C16M', 'C24M', 'C30M', 'D30', 'D35', 'D40',
               'D60', 'D70', 'geringe tragfähigkeit', 'mittlere tragfähigkeit',
               'hohe tragfähigkeit']

In [1650]:
test['Combined Sorting Class'] = ''
for i in test.index:
    test['Combined Sorting Class'].loc[i] = []
    for sort_class in sort_classes:
        pat = r' {} |^{} | {}$| {}\;'.format(sort_class.lower(), sort_class.lower(), sort_class.lower(), sort_class.lower())
        if len(re.findall(pat, test['Combined'].loc[i].lower()))>0 and sort_class.lower() not in ', '.join(test['Combined Sorting Class'].loc[i]):
            test['Combined Sorting Class'].loc[i].append(sort_class)
       
    try:
        test['Combined Sorting Class'].loc[i] = ', '.join(test['Combined Sorting Class'].loc[i])
    except:
        pass
test['Combined Sorting Class'] =  test['Combined Sorting Class'].apply(lambda x: np.nan if x==' ' or x=='' else x )
test['Combined Sorting Class'].value_counts()

hohe tragfähigkeit    2
S10                   2
Name: Combined Sorting Class, dtype: int64

## Main parameters

### Wood Types

In [1652]:
woods = ['Ahorn', 'Tanne', 'Sperrholz', 'Nussbaum', 'Erle',
        'Bongossi (Azobé)', 'Birke', 'Buche', 'Douglasie',
        'Edelkastanie', 'Eiche', 'Elsbeere', 'Erle', 'Esche', 'Fichte',
        'Kiefer', 'Kirschbaum', 'Lärche', 'Linde', 'Mahagoni', 'Teak',
        'Pappel', 'Robinie', 'Tanne', 'Ulme', 'Walnuss', 'Weide', 'Birne',
        'Fichte/Tanne', 'Fichte/Kiefer']

In [1653]:
test['Meta Wood'] = ''
for i in test.index:
    for wood in woods:
        pat = r' {} |^{} | {}$'.format(wood.lower(), wood.lower(), wood.lower())
        if len(re.findall(pat, test['Combined'].loc[i].lower()))>0:
            test['Meta Wood'].loc[i] = wood

In [1654]:
test['Meta Wood'] = test['Meta Wood'].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [1655]:
test['Wood Type'] = test['Material-T1'].fillna(test['Holzart-T1']).fillna(test['Holzart-Dekor-T1']).fillna(test['Meta Wood'])

#### Cheking wrong typs and woods

In [1656]:
wrong_wood = ['Holzwerkstoff (MDF)', 'Glas', 'Aluminium', 'Metall', 'Granit', 'Polymere', 'Granito',
              'Kunststoff','Laminiert','Gipskarton, Furniert','PVC','Beschichtet', 'Melaminbeschichtet' ,
              'Furniert','PVC-freie Werkstoffmischung',  'Foliert',  'Laminiert', 'Polymere','Polycarbonat',
              'Polyvinylchlorid (PVC)','Gipskarton','German Compact Composite (GCC)','Bandstahl','Foliert',
              'Faseprofil','Metall, Kunststoff', 'Textil, Kunststoff','Rotbuche','Multiplex',
              'Qualitätsspanplatte mit HPL-Beschichtung Profil aus Hart-PVC','Holz-Kunststoff-Verbundwerkstoffe (WPC)'
              ,'Beschichtete Flachpressplatte', 'Hart-PVC-Leiste','Lackiert','WPC', 
              'Weich-PVC','Unbehandelt','Kunststoffbeschichtet','Kunststoffummantelter Hartfaserkern',
               'Stahl','Beschichtete Flachpressplatte, Hart-PVC-Leiste','Beidseitig mit HPL beschichtete Spanplatte',
              'Polycarbonat', 'German Compact Composite (GCC)', 
                'Foliert', 'WPC', 'Multiplex', 'Beschichtet', 'Stahl']


In [1657]:
# Percentage of wrong wood types
test[(test['Wood Type'].isin(wrong_wood))].shape[0]/test.shape[0]*100

17.02883409945675

In [1658]:
test[(test['Wood Type'].isin(wrong_wood))]['Wood Type'].value_counts()

WPC           721
Kunststoff     82
Stahl           7
Aluminium       5
Name: Wood Type, dtype: int64

In [1659]:
test = test[(~test['Wood Type'].isin(wrong_wood))]
test.shape

(3971, 56)

In [1660]:
test['Wood Type'] = test['Wood Type'].str.lower()

### Surface

In [1661]:
strong_surfaces = ['geschliffen', 'gehobelt', 'gefast',
                   'sägerau', 'geriffelt', 'glatt gehobelt', 'glatte', 'gefräst',
                   'gebürstet', 'strukturiert', '1-Seitig gehobelt',
                   '2-Seitig gehobelt', '3-Seitig gehobelt', '4-Seitig gehobelt',
                   'genutet', 'unbehandelt', 'naturbelassen', 
                   'deckend', 'lasiert', 'vorgeölt', 'kesseldruckimprägniert (KDI)',
                   'hitzebehandelt', 'thermobehandelt', 'wachsen', 'imprägniert',
                   'Scharfkantig']

weak_surfaces = ['roh', 'keine']

In [1662]:
test['Combined Surface Treat'] = ''
for i in test.index:
    test['Combined Surface Treat'].loc[i] = []
    for surface in weak_surfaces:
        pat = r' {} |^{} | {}$'.format(surface.lower(), surface.lower(), surface.lower())
        if len(re.findall(pat, test['Combined'].loc[i].lower()))>0 and surface.lower() not in ', '.join(test['Combined Surface Treat'].loc[i]):
            test['Combined Surface Treat'].loc[i].append(surface)
    
    for surface in strong_surfaces:
        if surface.lower() in test['Combined'].loc[i].lower() and surface.lower() not in ', '.join(test['Combined Surface Treat'].loc[i]):
            test['Combined Surface Treat'].loc[i].append(surface)
    try:
        test['Combined Surface Treat'].loc[i] = ', '.join(test['Combined Surface Treat'].loc[i])
    except:
        pass

In [1663]:
test['Combined Surface Treat'] = test['Combined Surface Treat'].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [1664]:
test['Surface'] = test['Oberflächenbehandlung-T1'].fillna(test['Oberfläche-T1']).fillna(test['Combined Surface Treat'])

### Drying method

In [1666]:
dry_method = ['technischer trocknung','technisch getrocknet', 'kammer getrocknet',
               'kammertrocken', 'AD (luftgetrocknet)',
               'KD (Künstlich getrocknet)']

In [1667]:
test['Combined Drying Method'] = ''
for i in test.index:
    test['Combined Drying Method'].loc[i] = []
    for method in dry_method:
        if method.lower() in test['Combined'].loc[i].lower() and method.lower() not in ', '.join(test['Combined Drying Method'].loc[i]):
            test['Combined Drying Method'].loc[i].append(method)
        
    try:
        test['Combined Drying Method'].loc[i] = ', '.join(test['Combined Drying Method'].loc[i])
    except:
        pass

In [1668]:
test['Combined Drying Method'] =  test['Combined Drying Method'].apply(lambda x: np.nan if x==' ' or x=='' else x )

In [1669]:
test['Combined Drying Method'].value_counts()

technisch getrocknet    3
Name: Combined Drying Method, dtype: int64

### Keywords

In [1670]:
from df2gspread import gspread2df as g2d
from oauth2client.service_account import ServiceAccountCredentials
spreadsheet_key = '1iDGux_PxTPSIkxnljeEhMifJo5jAXDhxOXLB8zlrOKg'
scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/macbook/Downloads/gs-credentials.json', scope) 
keywords = g2d.download(gfile=spreadsheet_key, wks_name = 'Keywords', credentials=credentials, col_names=True, row_names=True)

In [1671]:
rename_cols = {'Produktart':'Product Type', 
               'Holztyp':'Wood Type',
               'Qualität':'Quality',
               'Oberflächenqualität':'Surface Quality',
               'Sortierklasse':'Sorting Class',
               'Trocknung':'Drying Method'}
keywords = keywords.rename(columns=rename_cols)

In [1462]:
keywords.head(1)

,Product Type,Wood Type,Drying Method,Quality,Sorting Class,Surface Treatment,Surface Quality,Schnittklasse,Besondere Ende
1,Baumscheibe,Ahorn,technisch getrocknet,A-Sortierung,S7,geschliffen,NSI,S,gespitzt


In [1463]:
for col in keywords.columns:
    keywords[col] = keywords[col].apply(lambda x: np.nan if x == '' or x == ' ' or x== '-777' else x)

In [2051]:
#keywords['Wood Type'].loc[31] = 'Holz'

### Updating Product types

In [1672]:
test['Product Type'] = test['Art-T1'].copy()

In [1673]:
test['Product Type New'] = np.nan
test['Product Type New'].loc[0] = ''

In [1674]:
for i in matched_df.index:
    test['Product Type New'].loc[i] = []
    for key_word in keywords['Product Type'].unique():
        if key_word.lower() in test['Combined'].loc[i].lower():
            test['Product Type New'].loc[i].append(key_word)
    test['Product Type New'].loc[i] = ', '.join(test['Product Type New'].loc[i])

In [1675]:
test['Product Type'] = test['Product Type'].fillna(test['Product Type New'])

In [1735]:
wrong_products = ['Befestigungssysteme', 'Zubehör']

In [1737]:
test = test[~test['Product Type'].isin(wrong_products)]
test.shape

(3022, 63)

In [1739]:
#test['Product Type'].value_counts()

### Matching keywords

In [1676]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [1679]:
ren_cols = {#'Product Type New':'Product Type',
           'Combined Sorting Class':'Sorting Class',
           'Combined Quality':'Quality',
            'Combined Drying Method':'Drying Method',
            'Combined Surface Treat':'Surface Treatment',
            'Combined Surface Quality':'Surface Quality'}

In [1680]:
test = test.rename(columns=ren_cols)

In [1681]:
main_cols = ['URL','SKU','Product Type','Wood Type','Surface Treatment',
             'Drying Method','Sorting Class','Quality','Combined'
            #'Header','Surface Quality',
            ]

In [1682]:
test['Matched Keywords'] = np.nan
test['Other Keywords'] = np.nan
test = test.fillna('-777')
keywords = keywords.fillna('-999')

In [1683]:
%%time
def get_keywords(row):
    row['Matched Keywords'] = {}
    row['Other Keywords'] = {}
    
    matched_cols=[]
    for col in ['Product Type','Wood Type','Surface Treatment',#'Surface Quality',
                'Drying Method','Sorting Class','Quality']:
        
        for key_word in keywords[col].unique():
            if fuzz.token_set_ratio(str(row[col]).lower(), key_word.lower()) > 90 or key_word.lower() in str(row[col]):
                row['Matched Keywords'][row[col]] = key_word
                matched_cols.append(col)
    
    if len(matched_cols)==0:
        row['Other Keywords'] = np.nan
        row['Matched Keywords'] = np.nan
    
    for col in ['Product Type','Wood Type','Surface Treatment',#'Surface Quality',
                'Drying Method','Sorting Class','Quality']:
        
        if col not in matched_cols and len(matched_cols)>0 and row[col]!=-999:
            row['Other Keywords'][col] = row[col]
    
    return row
test = test.apply(get_keywords, axis=1)

CPU times: user 56.7 s, sys: 428 ms, total: 57.1 s
Wall time: 58.8 s


In [1684]:
test[['Matched Keywords','Other Keywords']].isna().sum()/test.shape[0]*100

Matched Keywords    0.0
Other Keywords      0.0
dtype: float64

### Cleaning NaNs



In [1685]:
for col in test.columns:
    test[col] =  test[col].apply(lambda x: np.nan if x =='' or x == 'nan' or x == ' ' or x == '-777' or x == '-999' else x)

In [1733]:
test[(test['Matched Keywords'].isna())].shape

(0, 63)

In [1687]:
test[(test['Matched Keywords'].isna()) & 
          (test['Wood Type']=='Holz')][main_cols].shape

(0, 9)

In [1688]:
test[(~test['Matched Keywords'].isna()) #&
          ]['Product Type'].nunique()#value_counts()

61

### Pre-cleaning

In [1689]:
for col in test.columns:
    test[col] = test[col].apply(lambda x: np.nan if x == '' or x == ' ' or x=='-999' or x=='-777' or x=='nan'else x)

# GS

In [1614]:
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
scope = ['https://spreadsheets.google.com/feeds'] 
credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/macbook/Downloads/gs-credentials.json', scope) 
gc = gspread.authorize(credentials)
spreadsheet_key = '1iDGux_PxTPSIkxnljeEhMifJo5jAXDhxOXLB8zlrOKg'

In [1691]:
sheets_clean.head(1)

,URL,Art-T1,Material-T1,Stärke-T1,Breite-T1,Länge-T1,Farbe-T1,Oberflächenbehandlung-T1,Gewicht-T2,Höhe-T2,Breite-T2,Tiefe-T2,Title,SKU,Description,Price Raw,Delivery,Klicksystem-T1,Größe (LxB)-T1,Gewicht-T1,Höhe-T1,Tiefe-T1,Farbton-T1,Optik-T1,Nutzungsklasse-T1,Trittschalldämmung-T1,Fußbodenheizung geeignet-T1,Dekor-Nachbildung-T1,Kantenform-T1,Holzart-T1,Imprägniert-T1,Güteklasse-T1,Einsatzbereich-T1,Oberfläche-T1,Zubehör-T1,Holzart-Dekor-T1,Anwendungszweck-T1,Title-Dims,Header-Dims-Mult,H-Dim1,H-Dim2,H-Dim3,Stärke-T1-Mult,Breite-T1-Mult,Länge-T1-Mult,Höhe-T2-Mult,Breite-T2-Mult,Tiefe-T2-Mult,Stärke-T1-mm,Breite-T1-mm,Länge-T1-mm,Höhe-T2-mm,Breite-T2-mm,Tiefe-T2-mm,Combined,Quality,Sorting Class,Meta Wood,Wood Type,Surface Treatment,Surface,Drying Method,Product Type New,Product Type,Matched Keywords,Other Keywords,Delivery Status,Delivery Yes/No
0,https://www.obi.de/terrassendielen/unterkonstr...,Unterkonstruktion,Douglasie,"4,5 cm",7 cm,200 cm,Natur,Unbehandelt,"3,20 kg","4,5 cm","2,00 m","7,0 cm","Unterkonstruktion Douglasie 4,5 cm x 7 cm x 20...",Art.Nr. 7239882,Ideal geeignet für Douglasie-Terrassen| Dauerh...,"Menge| 6,24| | €*| 3,12 € / m| inkl. gesetzl...",Online bestellen & liefern lassen| ca. 4-6 Wer...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"4,5cmx7cmx200cm",10,4.5,7,200,10.0,10.0,10.0,10.0,1000.0,10.0,45.0,70.0,2000.0,45.0,2000.0,70.0,"Unterkonstruktion; Douglasie; 4,5 cm; 7 cm; 20...",NaN,NaN,Douglasie,Douglasie,unbehandelt,Unbehandelt,NaN,NaN,Unterkonstruktion,"{'Douglasie': 'Douglasie', 'unbehandelt': 'unb...","{'Product Type': 'Unterkonstruktion', 'Drying ...",ca. 4-6 Werktage Lieferzeit,Yes


In [1719]:
sheets_clean = test[~test['Matched Keywords'].isna()]
sheets_clean.shape

(3971, 63)

In [1757]:
sheets_clean = sheets_clean[(~sheets_clean.duplicated(subset='URL', keep='last'))]
sheets_clean.shape

(3019, 16)

In [1721]:
sheets_clean['Website Name'] = 'www.obi.de'

In [1726]:
ren_cols = {'Title':'Product Title',
            'Breite-T1-mm':'Width (mm)',
            'Länge-T1-mm':'Length (mm)', 
            'Stärke-T1-mm':'Thickness (mm)',
            'Price':'Price (Euro)'}
sheets_clean = sheets_clean.rename(columns = ren_cols)

cols = ['Website Name', 'URL','Product Title','Product Type', 'Wood Type', 'Width (mm)', 'Length (mm)', 'Thickness (mm)', 
        'Price (Euro)','Delivery Yes/No','Quality', 'Surface Treatment','Drying Method', #'Surface Quality',
        'Sorting Class','SKU','Delivery Status' ]

sheets_clean = sheets_clean[cols].rename(columns = ren_cols)

In [1754]:
sheets_clean = sheets_clean[~sheets_clean['Product Type'].isin(wrong_products)]

In [1741]:
sheets_clean[(sheets_clean['Width (mm)'].isna()) &
             (sheets_clean['Length (mm)'].isna()) &
             (sheets_clean['Thickness (mm)'].isna())].shape[0]/sheets_clean.shape[0]*100

15.998675057966214

In [1755]:
d2g.upload(sheets_clean.fillna('-'),
           spreadsheet_key,
           'OBI Clean Data',
           credentials=credentials,
           col_names=True,
           row_names=False,
           start_cell = 'A1',
           clean=True)

<Worksheet 'OBI Clean Data' id:667286089>